## Урок 14. Модель BERT и GPT
## Практическое задание

Задание

1. взять данные из https://www.kaggle.com/datasets/mrapplexz/bashim-quotes обучить модель GPT для генерации своих цитат


2. взять новостные данные из https://github.com/natasha/corus load_lenta2 нам понадобиться сам текст и заголовок обучить модель T5/ или GPT для генерации заголовков для статей

In [1]:
import pandas as pd
import random

In [2]:
!pip install transformers sentencepiece --quiet

## Задание 1

In [3]:
DATA_PATH = '/content/dataset.jsonl'

with open(DATA_PATH) as f:
     df = pd.read_json(DATA_PATH, lines=True).set_index('id')
df.head(3)

,date,rating,text
id,,,
1,2004-08-30 11:24:00+00:00,22010.0,"<Ares> ppdv, все юниксы очень дружелюбны.. они..."
2,2004-08-30 11:25:00+00:00,25105.0,<томатик_рад> а ты не чувствуешь красоту мира?...
3,2004-08-30 11:27:00+00:00,7192.0,"<Дор> ""мышка, почему у тебя такие большие глаз..."


In [4]:
import re

def clear_text(text):
    clr_text = re.sub(r"<.*?>", " ", text).lower()
    clr_text = summary = re.sub(r"\s", " ", clr_text)
    return clr_text

In [5]:
df["clear_text"] = df["text"].apply(lambda x: clear_text(x))
df

,date,rating,text,clear_text
id,,,,
1,2004-08-30 11:24:00+00:00,22010.0,"<Ares> ppdv, все юниксы очень дружелюбны.. они...","ppdv, все юниксы очень дружелюбны.. они прос..."
2,2004-08-30 11:25:00+00:00,25105.0,<томатик_рад> а ты не чувствуешь красоту мира?...,а ты не чувствуешь красоту мира? честно го...
3,2004-08-30 11:27:00+00:00,7192.0,"<Дор> ""мышка, почему у тебя такие большие глаз...","""мышка, почему у тебя такие большие глаза?"" ..."
4,2004-08-30 11:28:00+00:00,29169.0,"<PPDV[os2]> ""Мальчики, вы что больные, бегать ...","""мальчики, вы что больные, бегать в палату к..."
5,2004-08-30 11:26:00+00:00,7140.0,<Ohtori_Akio> мы - как разработчики - живём с ...,мы - как разработчики - живём с субейзом под...
...,...,...,...,...
463644,2020-11-26 06:12:00+00:00,698.0,"xxx: угадайте не гугля, что такое жопиздан!\ny...","xxx: угадайте не гугля, что такое жопиздан! yy..."
463645,2020-11-26 06:12:00+00:00,816.0,"xxx:\nПосетила шальная мысль заняться собой, ж...","xxx: посетила шальная мысль заняться собой, жи..."
463646,2020-11-26 06:46:00+00:00,88.0,#всебожьяроса\nxxx: Судьба айтишников вообще н...,#всебожьяроса xxx: судьба айтишников вообще не...


In [6]:
data = df.loc[:, 'clear_text']

In [7]:
data

id
1           ppdv, все юниксы очень дружелюбны.. они прос...
2           а ты не чувствуешь красоту мира?   честно го...
3           "мышка, почему у тебя такие большие глаза?" ...
4           "мальчики, вы что больные, бегать в палату к...
5           мы - как разработчики - живём с субейзом под...
                                ...                        
463644    xxx: угадайте не гугля, что такое жопиздан! yy...
463645    xxx: посетила шальная мысль заняться собой, жи...
463646    #всебожьяроса xxx: судьба айтишников вообще не...
463647    прибывшие на место правоохранители установили,...
463648    ххх: вот ребята из английской фирмы tvr (давны...
Name: clear_text, Length: 81497, dtype: object

In [8]:
text=df['clear_text']

In [9]:
from sklearn.model_selection import train_test_split

def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        summary = str(texts).strip()
        data += summary + "  "
    f.write(data)

In [10]:
train, test = train_test_split(data, test_size=0.15)

build_text_files(train, 'train_dataset.txt')
build_text_files(test, 'test_dataset.txt')

In [11]:
str(len(train))

'69272'

In [12]:
str(len(test))

'12225'

In [13]:
train

id
65317       соседи глохнут под - неизвестный исполнитель...
8945        я б завела животинку(   но кв снимается >_< ...
446193    в свете новых технологий и любви к лайкам, реп...
458293    xxx: новый взгляд на народную мудрость xxx: у ...
322173    fagot_nwe: у меня товарищ пришел домой обкурен...
                                ...                        
62373     звонок мне из тернополя (т): т: у нас неустран...
426502    ххх: купил видеорегистратор в надежде снять ка...
429837    работаю около загса и, соответственно, почти е...
410371    ххх: вот не понимаю, почему выпускников фабрик...
454478    форум с разделом "опознание игр":  xxx: ...сам...
Name: clear_text, Length: 69272, dtype: object

In [14]:
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForCausalLM

import torch
device = torch.device("cuda")

In [15]:
# Загружаем large GPT3, которая основана на GPT2
model_name = 'sberbank-ai/rugpt3large_based_on_gpt2'
#model_name = 'Grossmend/rudialogpt3_medium_based_on_gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [17]:
# Tuning model

training_args = TrainingArguments(

    "phrase",
    evaluation_strategy = "epoch",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=2,
    learning_rate=1e-5,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
    
    )

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [19]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 36002
  Num Epochs = 2
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 72004


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: ignored

остановил обучение, 10 часов предстояло ждать, а с большим размером батчей забивалась оперативка видеокарты.

In [19]:
sep = '\n***\n'  # Признак того, что твит закончился и нужногенерировать ещё один твит
# sep = '\n27479153	Sandy_mustache	2021-02-18 16:44:00	'


# Так как мы постоянно сэмплируем разные твиты,
# мы будем постоянно получать разное распределение
prefix = sep.join([''] + random.sample(list(text), k=5) + [''])

tokens = tokenizer(prefix, return_tensors='pt')
tokens = {k: v.to(model.device) for k, v in tokens.items()}
end_token_id = tokenizer.encode('***')[0]  # '***' - токен который будет оканчивать твит

# выводим то, что мы передаём на вход
print(prefix)


***
dago: у нас в офисе новое растение grey: еще одного бухгалтера взяли?
***
--> lizaveta has joined this channel (679@62.105.15.72).   эй ей   здраствуйте лизочка   скажите, какого числа вы родились?   17   какого месяца?   06   какого года?   1987   какого хуя?   ??????????????????????
***
функция wolf() в полнолуние void на луну!
***
vision: в общем, если я не могу заставить организм спать, ща пойду мыть посуду vision: а то там небось уже тараканы в царя горы играют
***
xxx: знакомые рссказывали. мужик садился в машину на гаражах. куда-то запропастился навесной замок, он второпях просто замотал дверь проволокой и уехал. ночью воры вскрыли несколько гаражей - три влево и три вправо от гаража этого мужика. его гараж - с проволокой вместо замка - не тронули, а на дверях красовалась надпись баллончиком "суперзамки не взламываем"
***



In [20]:
# Генерируем какой то осмысленный текст похожий на твит.

size = tokens['input_ids'].shape[1]
output = model.generate(
    **tokens, 
    #end_token=end_token_id,
    do_sample=False,  # вкл/выкл режим выдачи нескольких вариантов д.б. ещё один параметр
    max_length=size+128, 
    repetition_penalty=4.2,  # штраф за повторы одинарных токенов
    temperature=0.7,  # температура
    num_beams=10,  # Строим дерево глубины 10
    # no_repeat_ngram_size=3  # ! тройки подряд идущих токенов не должны повторяться (3 и меньше токенов не должны повторяться)
)

decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


uri_loskutov: вот что бывает, когда едешь со скоростью 100 км/ч по городу...
***
sergey_petrovich: как же мне надоели эти тупые блондинки!!!!!
***
maximilianus: ААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААА


In [21]:
# Генерируем какой то осмысленный текст похожий на твит.

size = tokens['input_ids'].shape[1]
output = model.generate(
    **tokens, 
    #end_token=end_token_id,
    do_sample=False,  # вкл/выкл режим выдачи нескольких вариантов д.б. ещё один параметр
    max_length=size+128, 
    repetition_penalty=4.2,  # штраф за повторы одинарных токенов
    temperature=0.7,  # температура
    num_beams=5,  # Строим дерево глубины 10
    # no_repeat_ngram_size=3  # ! тройки подряд идущих токенов не должны повторяться (3 и меньше токенов не должны повторяться)
)

decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


oung_cat: даааа... это ж надо было додуматься до такой хуйни!!!
***
sergey_petrovich: как же мне нравится эта фраза из анекдота про блондинку и брюнетку :)))))
***
maximilianus: ооооойййййййййййййййййййййййййййййййййййййййййййййййййййййййййййй


Ну вполне себе цитаты)) хоть и не дообучал сеть

## Задание 2

In [22]:
!pip install datasets --quiet

     |████████████████████████████████| 365 kB 25.2 MB/s 
     |████████████████████████████████| 212 kB 70.4 MB/s 
     |████████████████████████████████| 115 kB 64.9 MB/s 
     |████████████████████████████████| 127 kB 65.6 MB/s 


In [23]:
from datasets import load_dataset

dataset_train = load_dataset('IlyaGusev/gazeta', revision="v1.0", split= 'train[:10%]')
dataset_test = load_dataset('IlyaGusev/gazeta', revision="v1.0", split= 'test[:10%]')

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/52400 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5770 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5265 [00:00<?, ? examples/s]

Dataset gazeta downloaded and prepared to /root/.cache/huggingface/datasets/IlyaGusev___gazeta/default/1.0.0/ef9349c3c0f3112ca4036520d76c4bc1b8a79d30bc29643c6cae5a094d44e457. Subsequent calls will reuse this data.


In [24]:
dataset_train

Dataset({
    features: ['text', 'summary', 'title', 'date', 'url'],
    num_rows: 5240
})

In [25]:
dataset_test

Dataset({
    features: ['text', 'summary', 'title', 'date', 'url'],
    num_rows: 577
})

In [27]:
dataset_test['summary'][0]

'В NASA назвали четыре миссии в дальний космос, которые в этом десятилетии могут быть запущены американцами. Среди них — две миссии по изучению Венеры, полет к спутнику Юпитера и экспедиция к Тритону, спутнику Нептуна.'

In [26]:
dataset_test['title'][0]

'Венера, Ио или Тритон: куда полетит NASA'

In [28]:
model_name = "IlyaGusev/rut5_base_sum_gazeta"

In [29]:
def len_tok(text):
    return len(text.split())

In [30]:
max_len_sum, max_len_tl = max(map(len_tok, dataset_train['summary'])), max(map(len_tok, dataset_train['title']))
max_len_sum, max_len_tl

(75, 18)

In [31]:
max_len_sum, max_len_tl = 60, 15

In [32]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    tokenized_input = tokenizer(batch['summary'], padding='max_length', truncation=True, max_length=max_len_sum)
    tokenized_label = tokenizer(batch['title'], padding='max_length', truncation=True, max_length=max_len_tl)

    tokenized_input['labels'] = tokenized_label['input_ids']

    return tokenized_input

dataset_train = dataset_train.map(tokenize, batched=True, batch_size=8)
dataset_test = dataset_test.map(tokenize, batched=True, batch_size=8)

dataset_train.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])
dataset_test.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])

https://huggingface.co/IlyaGusev/rut5_base_sum_gazeta/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp7hlii3mf


storing https://huggingface.co/IlyaGusev/rut5_base_sum_gazeta/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/f2a7b6aa9f667eabdf0660f437c38ca43696ef49a86df2e264fc93750ab9abb5.30a9c79b7a80d5e1aa9c5b7d6aff517e04661218d78a71459b4587fa83baa33a
creating metadata file for /root/.cache/huggingface/transformers/f2a7b6aa9f667eabdf0660f437c38ca43696ef49a86df2e264fc93750ab9abb5.30a9c79b7a80d5e1aa9c5b7d6aff517e04661218d78a71459b4587fa83baa33a
https://huggingface.co/IlyaGusev/rut5_base_sum_gazeta/resolve/main/spiece.model not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmphoht66b5


storing https://huggingface.co/IlyaGusev/rut5_base_sum_gazeta/resolve/main/spiece.model in cache at /root/.cache/huggingface/transformers/9adebe2aa47a25febcf707ee15540f9e440f8d8e79697dae237fc4e6ccad5019.b846524fbcbf3cf81e2302f8087043922ca4c445b4016bf16e707f7e2240a3e6
creating metadata file for /root/.cache/huggingface/transformers/9adebe2aa47a25febcf707ee15540f9e440f8d8e79697dae237fc4e6ccad5019.b846524fbcbf3cf81e2302f8087043922ca4c445b4016bf16e707f7e2240a3e6
https://huggingface.co/IlyaGusev/rut5_base_sum_gazeta/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpflohtks6


storing https://huggingface.co/IlyaGusev/rut5_base_sum_gazeta/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/9ea40ad78f35f554a9c607b947515d4e6f2f62e3a6a1ec3fa957e727cddbb635.975331e2a6fea3fd5d8f528410d471fb0f6f16b82a34e658f7a0d5eda5061b99
creating metadata file for /root/.cache/huggingface/transformers/9ea40ad78f35f554a9c607b947515d4e6f2f62e3a6a1ec3fa957e727cddbb635.975331e2a6fea3fd5d8f528410d471fb0f6f16b82a34e658f7a0d5eda5061b99
https://huggingface.co/IlyaGusev/rut5_base_sum_gazeta/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpc9eo8e3z


storing https://huggingface.co/IlyaGusev/rut5_base_sum_gazeta/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/6ef3ae62c46b2a0173c15263e19f4443143979a9eefdfc1feab6a709496e7be2.294ebaa4cd17bb284635004c92d2c4d522ec488c828dcce0c2471b6f28e3fe82
creating metadata file for /root/.cache/huggingface/transformers/6ef3ae62c46b2a0173c15263e19f4443143979a9eefdfc1feab6a709496e7be2.294ebaa4cd17bb284635004c92d2c4d522ec488c828dcce0c2471b6f28e3fe82
loading file https://huggingface.co/IlyaGusev/rut5_base_sum_gazeta/resolve/main/spiece.model from cache at /root/.cache/huggingface/transformers/9adebe2aa47a25febcf707ee15540f9e440f8d8e79697dae237fc4e6ccad5019.b846524fbcbf3cf81e2302f8087043922ca4c445b4016bf16e707f7e2240a3e6
loading file https://huggingface.co/IlyaGusev/rut5_base_sum_gazeta/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/9ea40ad78f35f554a9c607b947515d4e6f2f62e3a6a1ec3fa957e727cddbb635.975331e2a6fea3fd5d8f528410d471fb0f6f16b

  0%|          | 0/655 [00:00<?, ?ba/s]

  0%|          | 0/73 [00:00<?, ?ba/s]

In [33]:
dataset_train.save_to_disk('gazeta/train')
dataset_test.save_to_disk('gazeta/test')

In [34]:
!ls

cached_lm_GPT2TokenizerFast_128_test_dataset.txt	gazeta
cached_lm_GPT2TokenizerFast_128_test_dataset.txt.lock	phrase
cached_lm_GPT2TokenizerFast_128_train_dataset.txt	sample_data
cached_lm_GPT2TokenizerFast_128_train_dataset.txt.lock	test_dataset.txt
dataset.jsonl						train_dataset.txt


In [35]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

model = T5ForConditionalGeneration.from_pretrained(model_name)

https://huggingface.co/IlyaGusev/rut5_base_sum_gazeta/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpmhc5s8dq


storing https://huggingface.co/IlyaGusev/rut5_base_sum_gazeta/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/426a325da473aa010e527ee99032b35ce9354913e38282d34e50dd75856c82f7.87df939950b4282d4195b92cd0a6209ec6d3d69e74b13a77d87890cf3a5ded7b
creating metadata file for /root/.cache/huggingface/transformers/426a325da473aa010e527ee99032b35ce9354913e38282d34e50dd75856c82f7.87df939950b4282d4195b92cd0a6209ec6d3d69e74b13a77d87890cf3a5ded7b
loading configuration file https://huggingface.co/IlyaGusev/rut5_base_sum_gazeta/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/426a325da473aa010e527ee99032b35ce9354913e38282d34e50dd75856c82f7.87df939950b4282d4195b92cd0a6209ec6d3d69e74b13a77d87890cf3a5ded7b
Model config T5Config {
  "_name_or_path": "cointegrated/rut5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "bos_token_id": 2,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 2,
  "dense_act_fn": "gel

storing https://huggingface.co/IlyaGusev/rut5_base_sum_gazeta/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/f4c11e29521f27cff5768a7f18580488bf450ce29ef1e4c8e06f9802e9d6ab42.e3b906d7892e5f268c9d2f19a84d52ddfe543929f0dca80cb66a59459d424a1b
creating metadata file for /root/.cache/huggingface/transformers/f4c11e29521f27cff5768a7f18580488bf450ce29ef1e4c8e06f9802e9d6ab42.e3b906d7892e5f268c9d2f19a84d52ddfe543929f0dca80cb66a59459d424a1b
loading weights file https://huggingface.co/IlyaGusev/rut5_base_sum_gazeta/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/f4c11e29521f27cff5768a7f18580488bf450ce29ef1e4c8e06f9802e9d6ab42.e3b906d7892e5f268c9d2f19a84d52ddfe543929f0dca80cb66a59459d424a1b
All model checkpoint weights were used when initializing T5ForConditionalGeneration.

All the weights of T5ForConditionalGeneration were initialized from the model checkpoint at IlyaGusev/rut5_base_sum_gazeta.
If your task is similar to the tas

In [36]:
output_dir = 'gazeta/output'

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_accumulation_steps=1, # Number of eval steps to keep in GPU (the higher, the mor vRAM used)
    prediction_loss_only=True, # If I need co compute only loss and not other metrics, setting this to true will use less RAM
    learning_rate=0.00001,
    evaluation_strategy='steps', # Run evaluation every eval_steps
    save_steps=1000, # How often to save a checkpoint
    save_total_limit=1, # Number of maximum checkpoints to save
    remove_unused_columns=True, # Removes useless columns from the dataset
    run_name='run_gazeta', # Wandb run name
    logging_steps=500, # How often to log loss to wandb
    eval_steps=500, # How often to run evaluation on the val_set
    logging_first_step=False, # Whether to log also the very first training step to wandb
    load_best_model_at_end=True, # Whether to load the best model found at each evaluation.
    metric_for_best_model="loss", # Use loss to evaluate best model.
    greater_is_better=False # Best model is the one with the lowest loss, not highest.
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [37]:
%%time

# Обучение. У нас 10 эпох.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 5240
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6550
/usr/local/lib/python3.7/dist-packages/transformers/data/data_collator.py:131: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  batch[k] = torch.tensor([f[k] for f in features])


Step,Training Loss,Validation Loss
500,8.256800,3.326211
1000,4.611500,3.158144
1500,2.722300,3.000120
2000,2.582800,2.911969
2500,2.499100,2.876213
3000,2.460400,2.857541
3500,2.422100,2.833924
4000,2.444400,2.816941
4500,2.390900,2.806586
5000,2.401000,2.794934


***** Running Evaluation *****
  Num examples = 577
  Batch size = 8
***** Running Evaluation *****
  Num examples = 577
  Batch size = 8
Saving model checkpoint to gazeta/output/checkpoint-1000
Configuration saved in gazeta/output/checkpoint-1000/config.json
Model weights saved in gazeta/output/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 577
  Batch size = 8
***** Running Evaluation *****
  Num examples = 577
  Batch size = 8
Saving model checkpoint to gazeta/output/checkpoint-2000
Configuration saved in gazeta/output/checkpoint-2000/config.json
Model weights saved in gazeta/output/checkpoint-2000/pytorch_model.bin
Deleting older checkpoint [gazeta/output/checkpoint-1000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 577
  Batch size = 8
***** Running Evaluation *****
  Num examples = 577
  Batch size = 8
Saving model checkpoint to gazeta/output/checkpoint-3000
Configuration saved in gazeta/output/checkpoint-3000/con

CPU times: user 25min 36s, sys: 2min 27s, total: 28min 3s
Wall time: 28min 54s


TrainOutput(global_step=6550, training_loss=3.0609561646439647, metrics={'train_runtime': 1733.3431, 'train_samples_per_second': 30.231, 'train_steps_per_second': 3.779, 'total_flos': 4174008606720000.0, 'train_loss': 3.0609561646439647, 'epoch': 10.0})

In [38]:
trainer.save_model(output_dir + '/model')

Saving model checkpoint to gazeta/output/model
Configuration saved in gazeta/output/model/config.json
Model weights saved in gazeta/output/model/pytorch_model.bin


In [39]:
INX = 100
print("SUMMARY: | {}".format(dataset_test['summary'][INX]))
print("TITLE: | {}".format(dataset_test['title'][INX]))

SUMMARY: | Российские фрегаты «Адмирал Макаров» и «Адмирал Григорович» с крылатыми ракетами «Калибр-НК» проходят через турецкие проливы Босфор и Дарданеллы в направлении Средиземного моря. Ранее эти корабли уже участвовали в военной операции РФ в Сирии и теперь направляются туда на фоне обострившейся ситуации в Идлибе.
TITLE: | Снова в Сирию: российские фрегаты идут в Средиземное море


In [40]:
device = "cuda"

In [49]:
import torch

input_text = dataset_test['summary'][INX]

with torch.no_grad():
    tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

    source_ids = tokenized_text['input_ids'].to(device, dtype = torch.long)
    source_mask = tokenized_text['attention_mask'].to(device, dtype = torch.long)

    generated_ids = model.generate(
        input_ids = source_ids,
        attention_mask = source_mask, 
        max_length=512,
        num_beams=10,
        temperature = 1.3,  
        repetition_penalty=1, 
        length_penalty=1, 
        early_stopping=True,
        no_repeat_ngram_size=2  # количество повторов n-грамм > 2 запрещено.
    )

    # Параметры подбираются эксперементально

    pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

print("\noutput:\n" + pred)


output:
Apple и Samsung подвергаются опасности


In [48]:
INX = 11
print("SUMMARY: | {}".format(dataset_test['summary'][INX]))
print("TITLE: | {}".format(dataset_test['title'][INX]))

input_text = dataset_test['summary'][INX]

with torch.no_grad():
    tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

    source_ids = tokenized_text['input_ids'].to(device, dtype = torch.long)
    source_mask = tokenized_text['attention_mask'].to(device, dtype = torch.long)

    generated_ids = model.generate(
        input_ids = source_ids,
        attention_mask = source_mask, 
        max_length=512,
        num_beams=10,
        temperature = 1.3,  
        repetition_penalty=1, 
        length_penalty=1, 
        early_stopping=True,
        no_repeat_ngram_size=2  # количество повторов n-грамм > 2 запрещено.
    )

    # Параметры подбираются эксперементально
    pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

print("\noutput:\n" + pred)

SUMMARY: | Юридическая компания из США подала в суд на Apple и Samsung за выпуск смартфонов, чье излучение якобы превышает установленные законом нормы. По версии истца, вендоры подвергают здоровье своих клиентов опасности. Между тем Федеральная комиссия по связи США провела собственное расследование уровня излучения популярных гаджетов и поделилась его результатами.
TITLE: | Опасное излучение: на Apple и Samsung подали в суд

output:
Apple и Samsung подвергают опасности


In [51]:
INX = 15
print("SUMMARY: | {}".format(dataset_test['summary'][INX]))
print("TITLE: | {}".format(dataset_test['title'][INX]))

input_text = dataset_test['summary'][INX]

with torch.no_grad():
    tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

    source_ids = tokenized_text['input_ids'].to(device, dtype = torch.long)
    source_mask = tokenized_text['attention_mask'].to(device, dtype = torch.long)

    generated_ids = model.generate(
        input_ids = source_ids,
        attention_mask = source_mask, 
        max_length=512,
        num_beams=10,
        temperature = 1.3,  
        repetition_penalty=1, 
        length_penalty=1, 
        early_stopping=True,
        no_repeat_ngram_size=2  # количество повторов n-грамм > 2 запрещено.
    )

    # Параметры подбираются эксперементально
    pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

print("\noutput:\n" + pred)

SUMMARY: | Для борьбы с с недобросовестными АЗС, недоливающими топливо, будут использоваться специальные приборы, а также проверки программного обеспечения. В этом вопросе Росстандарт намерен использовать опыт, полученный в сфере регулирования работы игровых автоматов.
TITLE: | Борьба с недоливом: заправки проверят скрытым прибором

output:
Росстандарт будет бороться с АЗС


In [52]:
INX = 20
print("SUMMARY: | {}".format(dataset_test['summary'][INX]))
print("TITLE: | {}".format(dataset_test['title'][INX]))

input_text = dataset_test['summary'][INX]

with torch.no_grad():
    tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

    source_ids = tokenized_text['input_ids'].to(device, dtype = torch.long)
    source_mask = tokenized_text['attention_mask'].to(device, dtype = torch.long)

    generated_ids = model.generate(
        input_ids = source_ids,
        attention_mask = source_mask, 
        max_length=512,
        num_beams=10,
        temperature = 1.3,  
        repetition_penalty=1, 
        length_penalty=1, 
        early_stopping=True,
        no_repeat_ngram_size=2  # количество повторов n-грамм > 2 запрещено.
    )

    # Параметры подбираются эксперементально
    pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

print("\noutput:\n" + pred)

SUMMARY: | Компания Apple намерена выпустить «бюджетный» смартфон iPhone SE 2 в марте следующего года. Этот гаджет станет преемником iPhone SE и обойдется покупателю не дороже $399 за базовую модель.
TITLE: | Новый дешевый iPhone: что готовит Apple

output:
Apple намерена выпустить iPhone SE


Вполне хорошие получаются заголовки. Классно варировать параметр num_beams=10, он сильно решает